In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

In [ ]:
memory = MemorySaver()

In [ ]:
class AgentState(TypedDict):
    messages : Annotated[list,add_messages]

In [ ]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode

search_tool = TavilySearch(max_results=2)
tools = [search_tool]
tool_node = ToolNode(tools)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def bot_node(state:AgentState):
    response = llm_with_tools.invoke(state['messages'])
    return {"messages":[response]}

In [ ]:
def should_continue(state:AgentState):
    if state['messages'][-1].tool_calls:
        return "tools"
    return END

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("bot",bot_node)
builder.add_node("tools",tool_node)
builder.add_edge(START,"bot")
builder.add_conditional_edges(
    'bot',
    should_continue,
    ['tools',END]
)
builder.add_edge("tools","bot")
graph = builder.compile(checkpointer=memory)


In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config={"configurable":{"thread_id":"1"}} # This is a configuration: thread_id represents the chat thread. This value will be unique for each new session.

response=graph.invoke({"messages":"Hi my name is Bigyan"},config=config)

response

In [ ]:
config={"configurable":{"thread_id":"1"}} # This is a configuration: thread_id represents the chat thread. This value will be unique for each new session.

response=graph.invoke({"messages":"What is my name?"},config=config)

response['messages'][-1].content

In [ ]:

config={"configurable":{"thread_id":"2"}} #here when change the thread id to another value where it has not got my name, it is going to output that it does'nt have my name. 

response=graph.invoke({"messages":"What is my name?"},config=config)

response['messages'][-1].content

In [ ]:

config={"configurable":{"thread_id":"2"}} 
response=graph.invoke({"messages":"Research about langgraph"},config=config)

response

### Output with streaming

In [ ]:
user_message = "What is a langchain?"
config = {"configurable":{"thread_id":"3"}}
events = graph.stream(
    {"messages":user_message},
    config=config,
    stream_mode="values"
)

In [ ]:
events

In [ ]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()